In [ ]:
import os
import json
import numpy as np
import cv2
from segment_anything import sam_model_registry, SamPredictor

# Paths
image_dir = '../Dataset/images/'
json_dir = '../Dataset/json_annotations/'
mask_dir = '../Dataset/sam_masks/'
os.makedirs(mask_dir, exist_ok=True)

# Load SAM model (ViT-H recommended)
sam_checkpoint = "sam_vit_h.pth"
model_type = "vit_h"
device = "cuda"  # or "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

# Generate masks from JSON prompts
for fname in os.listdir(image_dir):
    basename = os.path.splitext(fname)[0]
    img_path = os.path.join(image_dir, fname)
    json_path = os.path.join(json_dir, basename + '.json')

    if not os.path.exists(json_path):
        print(f"No annotation for {fname}, skipping.")
        continue

    image = cv2.imread(img_path)
    predictor.set_image(image)

    with open(json_path, 'r') as f:
        data = json.load(f)

    mask_combined = np.zeros(image.shape[:2], dtype=np.uint8)

    # Generate SAM masks for each region in your annotation
    for shape in data['shapes']:
        points = np.array(shape['points'])
        label = shape['label']

        # Use polygon centroid as prompt
        centroid = np.mean(points, axis=0)
        input_point = np.array([centroid])
        input_label = np.array([1])  # foreground

        masks, _, _ = predictor.predict(
            point_coords=input_point,
            point_labels=input_label,
            multimask_output=False
        )

        # Combine mask with unique value for each label
        if label == 'heart':
            mask_combined[masks[0]] = 1
        elif label == 'liver':
            mask_combined[masks[0]] = 2
        elif label == 'stomach':
            mask_combined[masks[0]] = 3

    # Save combined mask
    cv2.imwrite(os.path.join(mask_dir, basename + '_mask.png'), mask_combined * 80)  # scaled for clarity
    print(f"Mask created for {fname}")

print("SAM masks generated successfully.")
